In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import csv as csv
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from citipy import citipy
import requests
import json
from api_keys import apify_key

In [2]:
df_1=pd.read_csv("Resources/Milton_Avg_Rental_Price_updated.csv", encoding='ISO-8859-1')
df_2=pd.read_csv("Resources/Milton_Avg_Sale_Price_updated.csv", encoding='ISO-8859-1')
df_3=pd.read_csv("Resources/Milton_Median_Rental_Price_updated.csv", encoding='ISO-8859-1')
df_4=pd.read_csv("Resources/Milton_Median_Sale_Price_updated.csv", encoding='ISO-8859-1')
school_public_df = pd.read_csv('Resources/ODEF_v2_1.csv', encoding='ISO-8859-1')


In [3]:
milton_df_2021 = pd.read_csv("Resources/CensusProfile2021-ProfilRecensement2021-20231206084412.csv", encoding='ISO-8859-1')
milton_df_2021.head()

,Topic,Characteristic,Note,Total,Total_Flag,Total.1,Total_Flag.1
0,Population and dwellings,"Population, 2021",1.0,132979.0,NaN,NaN,...
1,Population and dwellings,"Population, 2016",1.0,110128.0,NaN,NaN,...
2,Population and dwellings,"Population percentage change, 2016 to 2021",NaN,20.7,NaN,20.7,NaN
3,Population and dwellings,Total private dwellings,2.0,41000.0,NaN,NaN,...
4,Population and dwellings,Private dwellings occupied by usual residents,3.0,40038.0,NaN,NaN,...


In [4]:
milton_df_2021.columns

Index(['Topic', 'Characteristic', 'Note', 'Total', 'Total_Flag', 'Total.1',
       'Total_Flag.1'],
      dtype='object')

In [5]:
milton_df_2021_cleaning = milton_df_2021.drop(milton_df_2021[milton_df_2021['Topic'] == 'Population and dwellings'].index[[2, 3, 4, 5, 6]])
milton_df_2021_cleaning

,Topic,Characteristic,Note,Total,Total_Flag,Total.1,Total_Flag.1
0,Population and dwellings,"Population, 2021",1.0,132979.0,NaN,NaN,...
1,Population and dwellings,"Population, 2016",1.0,110128.0,NaN,NaN,...
7,Age characteristics,Total - Age groups of the population - 100% data,NaN,132980.0,NaN,100.0,NaN
8,Age characteristics,0 to 14 years,NaN,31980.0,NaN,24.0,NaN
9,Age characteristics,0 to 4 years,NaN,8875.0,NaN,6.7,NaN
...,...,...,...,...,...,...,...
223,Labour force status,Unemployed,NaN,8035.0,NaN,NaN,...
224,Labour force status,Not in the labour force,NaN,28790.0,NaN,NaN,...
225,Labour force status,Participation rate,NaN,71.1,NaN,71.1,NaN
226,Labour force status,Employment rate,NaN,63.0,NaN,63.0,NaN


In [6]:
milton_df_2021_cleaning.loc[milton_df_2021_cleaning['Topic'] == 'Age characteristics'].index

Index([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
      dtype='int64')

In [7]:
indices_to_drop = [8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
milton_df_2021_cleaning = milton_df_2021_cleaning.drop(indices_to_drop, errors='ignore')

milton_df_2021_cleaning.head()

,Topic,Characteristic,Note,Total,Total_Flag,Total.1,Total_Flag.1
0,Population and dwellings,"Population, 2021",1.0,132979.0,NaN,NaN,...
1,Population and dwellings,"Population, 2016",1.0,110128.0,NaN,NaN,...
7,Age characteristics,Total - Age groups of the population - 100% data,NaN,132980.0,NaN,100.0,NaN
12,Age characteristics,15 to 64 years,NaN,87770.0,NaN,66.0,NaN
23,Age characteristics,65 years and over,NaN,13225.0,NaN,9.9,NaN


In [8]:
milton_df_2021_cleaning = milton_df_2021_cleaning.loc[milton_df_2021_cleaning['Topic'] != 'Household and dwelling characteristics']
milton_df_2021_cleaning.head()

,Topic,Characteristic,Note,Total,Total_Flag,Total.1,Total_Flag.1
0,Population and dwellings,"Population, 2021",1.0,132979.0,NaN,NaN,...
1,Population and dwellings,"Population, 2016",1.0,110128.0,NaN,NaN,...
7,Age characteristics,Total - Age groups of the population - 100% data,NaN,132980.0,NaN,100.0,NaN
12,Age characteristics,15 to 64 years,NaN,87770.0,NaN,66.0,NaN
23,Age characteristics,65 years and over,NaN,13225.0,NaN,9.9,NaN


In [9]:
milton_df_2021_cleaning.loc[milton_df_2021_cleaning['Topic'] == 'Income of households in 2020'].index


Index([ 57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
        99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115],
      dtype='int64')

In [10]:
indices_to_drop = [57, 60,  61,  62,  63,  64,  65,  66,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
        99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115]
milton_df_2021_cleaning = milton_df_2021_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2021_cleaning.head()

,Topic,Characteristic,Note,Total,Total_Flag,Total.1,Total_Flag.1
0,Population and dwellings,"Population, 2021",1.0,132979.0,NaN,NaN,...
1,Population and dwellings,"Population, 2016",1.0,110128.0,NaN,NaN,...
7,Age characteristics,Total - Age groups of the population - 100% data,NaN,132980.0,NaN,100.0,NaN
12,Age characteristics,15 to 64 years,NaN,87770.0,NaN,66.0,NaN
23,Age characteristics,65 years and over,NaN,13225.0,NaN,9.9,NaN


In [11]:
milton_df_2021_cleaning.loc[milton_df_2021_cleaning['Topic'] == 'Household characteristics'].index

Index([116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
       172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185,
       186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199,
       200, 201],
      dtype='int64')

In [12]:

milton_df_2021_cleaning

,Topic,Characteristic,Note,Total,Total_Flag,Total.1,Total_Flag.1
0,Population and dwellings,"Population, 2021",1.0,132979.0,NaN,NaN,...
1,Population and dwellings,"Population, 2016",1.0,110128.0,NaN,NaN,...
7,Age characteristics,Total - Age groups of the population - 100% data,NaN,132980.0,NaN,100.0,NaN
12,Age characteristics,15 to 64 years,NaN,87770.0,NaN,66.0,NaN
23,Age characteristics,65 years and over,NaN,13225.0,NaN,9.9,NaN
...,...,...,...,...,...,...,...
223,Labour force status,Unemployed,NaN,8035.0,NaN,NaN,...
224,Labour force status,Not in the labour force,NaN,28790.0,NaN,NaN,...
225,Labour force status,Participation rate,NaN,71.1,NaN,71.1,NaN
226,Labour force status,Employment rate,NaN,63.0,NaN,63.0,NaN


In [13]:
indices_to_drop = [119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
       172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185,
       186, 187, 192, 193, 194, 195, 196, 197, 198, 199,
       200, 201]
milton_df_2021_cleaning = milton_df_2021_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2021_cleaning

,Topic,Characteristic,Note,Total,Total_Flag,Total.1,Total_Flag.1
0,Population and dwellings,"Population, 2021",1.0,132979.0,NaN,NaN,...
1,Population and dwellings,"Population, 2016",1.0,110128.0,NaN,NaN,...
7,Age characteristics,Total - Age groups of the population - 100% data,NaN,132980.0,NaN,100.0,NaN
12,Age characteristics,15 to 64 years,NaN,87770.0,NaN,66.0,NaN
23,Age characteristics,65 years and over,NaN,13225.0,NaN,9.9,NaN
58,Income of households in 2020,Median total income of household in 2020 ($),NaN,126000.0,NaN,126000.0,NaN
59,Income of households in 2020,Median after-tax income of household in 2020...,NaN,107000.0,NaN,107000.0,NaN
67,Income of households in 2020,Average total income of household in 2020 ($),NaN,142600.0,NaN,142600.0,NaN
68,Income of households in 2020,Average after-tax income of household in 202...,NaN,116300.0,NaN,116300.0,NaN
116,Household characteristics,Total - Private households by tenure - 25% sam...,50.0,40035.0,NaN,100.0,NaN


In [14]:
milton_df_2021_cleaning = milton_df_2021_cleaning.loc[milton_df_2021_cleaning['Topic'] != 'Mobility status 1 year ago']
milton_df_2021_cleaning = milton_df_2021_cleaning.loc[milton_df_2021_cleaning['Topic'] != 'Mobility status 5 years ago']
milton_df_2021_cleaning

,Topic,Characteristic,Note,Total,Total_Flag,Total.1,Total_Flag.1
0,Population and dwellings,"Population, 2021",1.0,132979.0,NaN,NaN,...
1,Population and dwellings,"Population, 2016",1.0,110128.0,NaN,NaN,...
7,Age characteristics,Total - Age groups of the population - 100% data,NaN,132980.0,NaN,100.0,NaN
12,Age characteristics,15 to 64 years,NaN,87770.0,NaN,66.0,NaN
23,Age characteristics,65 years and over,NaN,13225.0,NaN,9.9,NaN
58,Income of households in 2020,Median total income of household in 2020 ($),NaN,126000.0,NaN,126000.0,NaN
59,Income of households in 2020,Median after-tax income of household in 2020...,NaN,107000.0,NaN,107000.0,NaN
67,Income of households in 2020,Average total income of household in 2020 ($),NaN,142600.0,NaN,142600.0,NaN
68,Income of households in 2020,Average after-tax income of household in 202...,NaN,116300.0,NaN,116300.0,NaN
116,Household characteristics,Total - Private households by tenure - 25% sam...,50.0,40035.0,NaN,100.0,NaN


In [15]:
milton_df_2021_cleaning.loc[milton_df_2021_cleaning['Topic'] == 'Labour force status'].index

Index([220, 221, 222, 223, 224, 225, 226, 227], dtype='int64')

In [16]:
indices_to_drop = [220, 221, 222, 223, 224, 225, 226]
milton_df_2021_cleaning = milton_df_2021_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2021_cleaning = milton_df_2021_cleaning.reset_index(drop=True)
milton_df_2021_cleaning

,Topic,Characteristic,Note,Total,Total_Flag,Total.1,Total_Flag.1
0,Population and dwellings,"Population, 2021",1.0,132979.0,NaN,NaN,...
1,Population and dwellings,"Population, 2016",1.0,110128.0,NaN,NaN,...
2,Age characteristics,Total - Age groups of the population - 100% data,NaN,132980.0,NaN,100.0,NaN
3,Age characteristics,15 to 64 years,NaN,87770.0,NaN,66.0,NaN
4,Age characteristics,65 years and over,NaN,13225.0,NaN,9.9,NaN
5,Income of households in 2020,Median total income of household in 2020 ($),NaN,126000.0,NaN,126000.0,NaN
6,Income of households in 2020,Median after-tax income of household in 2020...,NaN,107000.0,NaN,107000.0,NaN
7,Income of households in 2020,Average total income of household in 2020 ($),NaN,142600.0,NaN,142600.0,NaN
8,Income of households in 2020,Average after-tax income of household in 202...,NaN,116300.0,NaN,116300.0,NaN
9,Household characteristics,Total - Private households by tenure - 25% sam...,50.0,40035.0,NaN,100.0,NaN


In [17]:
milton_housing_df = pd.read_excel("Resources/TableExport.xlsx")
milton_housing_df

,Year,Single,Semi-Detached,Row,Apartment,All
0,1990,35,0,0,154,189
1,1991,16,0,0,0,16
2,1992,20,0,0,0,20
3,1993,27,0,0,0,27
4,1994,24,0,0,0,24
5,1995,24,0,0,0,24
6,1996,19,0,0,0,19
7,1997,14,0,0,0,14
8,1998,30,0,0,12,42
9,1999,41,0,0,51,92


In [18]:
milton_df_2016 = pd.read_csv("Resources/CensusProfile2016-ProfilRecensement2016-20231207084508.csv", encoding='ISO-8859-1')
milton_df_2016['Topic'].unique()

array(['Population and dwellings', 'Age characteristics',
       'Household and dwelling characteristics', 'Marital status',
       'Family characteristics', 'Household type',
       'Knowledge of official languages',
       'First official language spoken', 'Mother tongue',
       'Language spoken most often at home',
       'Other language spoken regularly at home',
       'Income of individuals in 2015', 'Income of households in 2015',
       'Income of economic families in 2015', 'Low income in 2015',
       'Knowledge of languages', 'Citizenship',
       'Immigrant status and period of immigration', 'Age at immigration',
       'Immigrants by selected place of birth',
       'Recent immigrants by selected places of birth',
       'Generation status', 'Admission category and applicant type',
       'Aboriginal population', 'Visible minority population',
       'Ethnic origin population', 'Household characteristics',
       ' Highest certificate; diploma or degree',
       'Major fi

In [19]:
milton_df_2016_cleaning = milton_df_2016.loc[milton_df_2016['Topic'] != 'Household and dwelling characteristics']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'First official language spoken']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Marital status']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Other language spoken regularly at home']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Family characteristics']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Knowledge of official languages']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Mother tongue']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Language spoken most often at home']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Income of individuals in 2015']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Income of economic families in 2015']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Low income in 2015']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Knowledge of languages']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Citizenship']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Immigrant status and period of immigration']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Age at immigration']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Immigrants by selected place of birth']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Recent immigrants by selected places of birth']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Generation status']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Admission category and applicant type']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Aboriginal population']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Visible minority population']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Ethnic origin population']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Highest certificate; diploma or degree']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Major field of study - Classification of Instructional Programs (CIP) 2016']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Location of study compared with province or territory of residence with countries outside Canada']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Occupation - National Occupational Classification (NOC) 2016']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Industry - North American Industry Classification System (NAICS) 2012']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Place of work status']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Commuting destination']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Main mode of commuting']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Commuting duration']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Time leaving for work']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Language used most often at work']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Other language used regularly at work']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Mobility status - Place of residence 1 year ago']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Mobility status - Place of residence 5 years ago']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Household type']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Work activity during the reference year']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Class of worker']
milton_df_2016_cleaning.head()

,Topic,Characteristics,Unnamed: 2,Total
0,Population and dwellings,Population; 2016,NaN,110128.0
1,Population and dwellings,Population; 2011,NaN,84362.0
2,Population and dwellings,Population percentage change; 2011 to 2016,NaN,30.5
3,Population and dwellings,Total private dwellings,NaN,35537.0
4,Population and dwellings,Private dwellings occupied by usual residents,NaN,34257.0


In [20]:
milton_df_2016_cleaning = milton_df_2016_cleaning.drop(milton_df_2016_cleaning[milton_df_2016_cleaning['Topic'] == 'Population and dwellings'].index[[1, 2, 3, 4, 5, 6]])
milton_df_2016_cleaning.head()

,Topic,Characteristics,Unnamed: 2,Total
0,Population and dwellings,Population; 2016,NaN,110128.0
7,Age characteristics,Total - Age groups and average age of the popu...,NaN,110125.0
8,Age characteristics,0 to 14 years,NaN,28405.0
9,Age characteristics,0 to 4 years,NaN,9315.0
10,Age characteristics,5 to 9 years,NaN,10505.0


In [21]:
milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] == 'Age characteristics'].index


Index([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
      dtype='int64')

In [22]:
indices_to_drop = [8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
milton_df_2016_cleaning = milton_df_2016_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2016_cleaning.head()

,Topic,Characteristics,Unnamed: 2,Total
0,Population and dwellings,Population; 2016,NaN,110128.0
7,Age characteristics,Total - Age groups and average age of the popu...,NaN,110125.0
12,Age characteristics,15 to 64 years,NaN,71580.0
23,Age characteristics,65 years and over,NaN,10145.0
740,Income of households in 2015,Total - Income statistics in 2015 for private ...,NaN,34255.0


In [23]:
milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] == 'Income of households in 2015'].index


Index([740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753,
       754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767,
       768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781,
       782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795,
       796, 797, 798],
      dtype='int64')

In [24]:

indices_to_drop = [740, 743, 744, 745, 746, 747, 748, 749, 752, 753,
       754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767,
       768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781,
       782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795,
       796, 797, 798]
milton_df_2016_cleaning = milton_df_2016_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2016_cleaning

,Topic,Characteristics,Unnamed: 2,Total
0,Population and dwellings,Population; 2016,NaN,110128.0
7,Age characteristics,Total - Age groups and average age of the popu...,NaN,110125.0
12,Age characteristics,15 to 64 years,NaN,71580.0
23,Age characteristics,65 years and over,NaN,10145.0
741,Income of households in 2015,Median total income of households in 2015 ($),NaN,104730.0
...,...,...,...,...
1867,Labour force status,Unemployed,NaN,3330.0
1868,Labour force status,Not in the labour force,NaN,20740.0
1869,Labour force status,Participation rate,NaN,74.2
1870,Labour force status,Employment rate,NaN,70.1


In [25]:
milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] == 'Household characteristics'].index


Index([1616, 1617, 1618, 1619, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627,
       1628, 1629, 1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639,
       1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651,
       1652, 1653, 1654, 1655, 1656, 1657, 1658, 1659, 1660, 1661, 1662, 1663,
       1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1675,
       1676, 1677, 1678, 1679, 1680, 1681],
      dtype='int64')

In [26]:
indices_to_drop = [1619, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627,
       1628, 1629, 1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639,
       1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651,
       1652, 1653, 1654, 1655, 1656, 1657, 1658, 1659, 1660, 1661, 1662, 1663,
       1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1677, 1678, 1679]
milton_df_2016_cleaning = milton_df_2016_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2016_cleaning.head()

,Topic,Characteristics,Unnamed: 2,Total
0,Population and dwellings,Population; 2016,NaN,110128.0
7,Age characteristics,Total - Age groups and average age of the popu...,NaN,110125.0
12,Age characteristics,15 to 64 years,NaN,71580.0
23,Age characteristics,65 years and over,NaN,10145.0
741,Income of households in 2015,Median total income of households in 2015 ($),NaN,104730.0


In [27]:
indices_to_drop = range (1682,1712)
milton_df_2016_cleaning = milton_df_2016_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2016_cleaning

,Topic,Characteristics,Unnamed: 2,Total
0,Population and dwellings,Population; 2016,NaN,110128.0
7,Age characteristics,Total - Age groups and average age of the popu...,NaN,110125.0
12,Age characteristics,15 to 64 years,NaN,71580.0
23,Age characteristics,65 years and over,NaN,10145.0
741,Income of households in 2015,Median total income of households in 2015 ($),NaN,104730.0
742,Income of households in 2015,Median after-tax income of households in 201...,NaN,88711.0
750,Income of households in 2015,Average total income of households in 2015 ($),NaN,120157.0
751,Income of households in 2015,Average after-tax income of households in 20...,NaN,97783.0
1616,Household characteristics,Total - Private households by tenure - 25% sam...,NaN,34260.0
1617,Household characteristics,Owner,NaN,29415.0


In [28]:
indices_to_drop = range (1864,1871)
milton_df_2016_cleaning = milton_df_2016_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2016_cleaning = milton_df_2016_cleaning.reset_index(drop=True)
milton_df_2016_cleaning

,Topic,Characteristics,Unnamed: 2,Total
0,Population and dwellings,Population; 2016,NaN,110128.0
1,Age characteristics,Total - Age groups and average age of the popu...,NaN,110125.0
2,Age characteristics,15 to 64 years,NaN,71580.0
3,Age characteristics,65 years and over,NaN,10145.0
4,Income of households in 2015,Median total income of households in 2015 ($),NaN,104730.0
5,Income of households in 2015,Median after-tax income of households in 201...,NaN,88711.0
6,Income of households in 2015,Average total income of households in 2015 ($),NaN,120157.0
7,Income of households in 2015,Average after-tax income of households in 20...,NaN,97783.0
8,Household characteristics,Total - Private households by tenure - 25% sam...,NaN,34260.0
9,Household characteristics,Owner,NaN,29415.0


In [29]:
milton_df = pd.merge(milton_df_2021_cleaning, milton_df_2016_cleaning, how='outer', left_index=True, right_index=True)
del milton_df['Unnamed: 2']
milton_df = milton_df.reset_index(drop=True)
milton_df.head()

,Topic_x,Characteristic,Note,Total_x,Total_Flag,Total.1,Total_Flag.1,Topic_y,Characteristics,Total_y
0,Population and dwellings,"Population, 2021",1.0,132979.0,NaN,NaN,...,Population and dwellings,Population; 2016,110128.0
1,Population and dwellings,"Population, 2016",1.0,110128.0,NaN,NaN,...,Age characteristics,Total - Age groups and average age of the popu...,110125.0
2,Age characteristics,Total - Age groups of the population - 100% data,NaN,132980.0,NaN,100.0,NaN,Age characteristics,15 to 64 years,71580.0
3,Age characteristics,15 to 64 years,NaN,87770.0,NaN,66.0,NaN,Age characteristics,65 years and over,10145.0
4,Age characteristics,65 years and over,NaN,13225.0,NaN,9.9,NaN,Income of households in 2015,Median total income of households in 2015 ($),104730.0


In [30]:
milton_df_one = pd.merge(df_1, df_2, how='inner', on='Primary Year')
milton_df_one = milton_df_one.rename(columns= {'Sale Price, Average_x' : 'Average Rental Price', 'Sale Price, Average_y' :'Average Housing Selling Price'})

In [31]:
milton_df_two = pd.merge(df_3, df_4, how='inner', on='Primary Year')
milton_df_two = milton_df_two.rename(columns= {'Sale Price, Median_x' : 'Median Rental Price', 'Sale Price, Median_y' :'Median Housing Selling Price'})
milton_df_two

,Primary Year,Median Rental Price,Median Housing Selling Price
0,2016,"$2,150","$621,000"
1,2017,"$2,100","$705,000"
2,2018,"$2,200","$675,000"
3,2019,"$2,300","$725,000"
4,2020,"$2,400","$810,786"
5,2021,"$2,650","$990,000"
6,2022,"$2,800","$1,100,000"
7,2023,"$2,900","$1,015,000"


In [32]:
milton_df_2 = pd.merge(milton_df_two, milton_df_one, how='inner', on='Primary Year')
milton_df_2


,Primary Year,Median Rental Price,Median Housing Selling Price,Average Rental Price,Average Housing Selling Price
0,2016,"$2,150","$621,000","$2,821","$657,525"
1,2017,"$2,100","$705,000","$2,341","$762,677"
2,2018,"$2,200","$675,000","$2,243","$717,413"
3,2019,"$2,300","$725,000","$2,369","$767,261"
4,2020,"$2,400","$810,786","$2,624","$880,169"
5,2021,"$2,650","$990,000","$2,785","$1,104,704"
6,2022,"$2,800","$1,100,000","$2,820","$1,196,104"
7,2023,"$2,900","$1,015,000","$2,989","$1,101,957"


In [33]:
school_df = school_public_df.loc[school_public_df['Provider'] == 'Province of Ontario']
school_df_milton = school_public_df.loc[school_public_df['CSDNAME'] == 'Milton']
school_df_milton


,Index,Source_ID,Facility_Name,Facility_Type,Authority_Name,ISCED010,ISCED020,ISCED1,ISCED2,ISCED3,...,Postal_Code,PRUID,CSDNAME,CSDUID,Longitude,Latitude,Geo_Source,Provider,CMANAME,CMAUID
266,56dfa4e43f9d3e1fd2c1,342322,École élémentaire Dyane-Adam,Public,CS Viamonde,1,1,1,0,0,...,L9T6J4,35,Milton,3524009,-79.8787,43.51839,Source,Province of Ontario,Toronto,535
358,8beeab446e576b16b529,694632,ÉÉC Saint-Nicolas,Catholic,CS catholique MonAvenir,1,1,1,0,0,...,L9T4A3,35,Milton,3524009,-79.8776,43.52873,Source,Province of Ontario,Toronto,535
363,6866c0a2daa890c0e715,779909,ÉÉC Sainte-Anne,Catholic,CS catholique MonAvenir,1,1,1,0,0,...,L9T7V8,35,Milton,3524009,-79.8434,43.51934,Source,Province of Ontario,Toronto,535
8215,553838317b31a9947fbe,669398,Montessori School of Milton Inc,Private,..,0,0,1,1,0,...,L9T7W6,35,Milton,3524009,-79.87822784,43.50116574,AGOL,Province of Ontario,Toronto,535
8339,02720786d966799ae311,668186,Oxford Learning Academy,Private,..,0,0,1,1,0,...,L9T3Z3,35,Milton,3524009,-79.87150069,43.52394302,AGOL,Province of Ontario,Toronto,535
8530,d192ddc575a774c16cba,669099,Stirling Montessori Academy Inc,Private,..,0,0,1,1,0,...,L9T1N4,35,Milton,3524009,-79.88537107,43.51116283,AGOL,Province of Ontario,Toronto,535
8564,2851b01e65a54c72206f,668379,Tarbiyah Elementary School,Private,..,0,0,1,1,0,...,L9T2T1,35,Milton,3524009,-79.87997186,43.52018415,AGOL,Province of Ontario,Toronto,535
9675,95ef9a36e7a8c3021369,689726,Bishop Paul Francis Reding Catholic Secondary ...,Catholic,Halton CDSB,0,0,0,1,1,...,L9T6H7,35,Milton,3524009,-79.8627,43.53094,Source,Province of Ontario,Toronto,535
9679,12776f7f8214dfa95775,708160,Guardian Angels Catholic Elementary School,Catholic,Halton CDSB,1,1,1,1,0,...,L9T6B1,35,Milton,3524009,-79.8437,43.51848,Source,Province of Ontario,Toronto,535
9683,db688946d6c58e93de3c,719005,Holy Rosary (Milton) Catholic Elementary School,Catholic,Halton CDSB,1,1,1,1,0,...,L9T2R3,35,Milton,3524009,-79.8856,43.51755,Source,Province of Ontario,Toronto,535


In [34]:
tax_df = pd.read_excel('Resources/Past-Years-Tax-Rates-for-Web (1).xlsx')


In [45]:
data = {
    'City': ['Milton', 'Milton'],
    'Year': ['2016', '2021'],
    'Population': [
        milton_df_2016_cleaning.at[0, 'Total'],
        milton_df_2021_cleaning.at[0, 'Total']
    ],
    'Household Income ($)': [
        milton_df_2016_cleaning.at[6, 'Total'],
        milton_df_2021_cleaning.at[7, 'Total']
    ],
    'Average Rent ($)': [
        milton_df_2.at[0, 'Average Rental Price'],
        milton_df_2.at[5, 'Average Rental Price']
    ],
    'Median Rent ($)': [
        milton_df_2.at[0, 'Median Rental Price'],
        milton_df_2.at[5, 'Median Rental Price']
    ],
    'Estimated House Value ($)': [
        milton_df_2016_cleaning.at[12, 'Total'],
        milton_df_2021_cleaning.at[15, 'Total']
    ],
    'Unemployment Rate (%)': [
        milton_df_2016_cleaning.at[15, 'Total'],
        milton_df_2021_cleaning.at[16, 'Total']
    ],
    'Property Tax Rate (%)': [
         tax_df.at[0,'Urban Tax Rate (no Enhanced waste)'],
         tax_df.at[21, 'Unnamed: 32']
     ],
    'Rented Houses': [
        milton_df_2016_cleaning.at[10, 'Total'],
        milton_df_2021_cleaning.at[11, 'Total']
    ],
    'Owned Houses': [
        milton_df_2016_cleaning.at[9, 'Total'],
        milton_df_2021_cleaning.at[10, 'Total']
    ],
    'New Built': [
        milton_housing_df.at[26, 'All'],
        milton_housing_df.at[31, 'All']
    ],
    'Average Sale Price ($)': [
        milton_df_2.at[0, 'Average Housing Selling Price'],
        milton_df_2.at[5, 'Average Housing Selling Price']
    ],
    'Median Sale Price ($)': [
        milton_df_2.at[0, 'Median Housing Selling Price'],
        milton_df_2.at[5, 'Median Housing Selling Price']
    ]
}

cleaned_milton_df = pd.DataFrame(data)

cleaned_milton_df = cleaned_milton_df.set_index(['City', 'Year'])
cleaned_milton_df

Population  Household Income ($) Average Rent ($)  \
City   Year                                                      
Milton 2016    110128.0              120157.0           $2,821   
       2021    132979.0              142600.0           $2,785   

            Median Rent ($)  Estimated House Value ($)  Unemployment Rate (%)  \
City   Year                                                                     
Milton 2016          $2,150                   622911.0                    5.6   
       2021          $2,650                  1022000.0                   11.4   

             Property Tax Rate (%)  Rented Houses  Owned Houses  New Built  \
City   Year                                                                  
Milton 2016               0.006820         4840.0       29415.0        891   
       2021               0.007449         6950.0       33085.0       1073   

            Average Sale Price ($) Median Sale Price ($)  
City   Year                                               
Milton 2016               $657,525              $621,000  
       2021             $1,104,704              $990,000

In [36]:
# Set the search parameters
target_city = "Milton, Canada"
radius = 10000

params = {
    "categories": 'leisure',
    'text': target_city,
    'apiKey': apify_key,
    'limit':100
}

base_url = "https://api.geoapify.com/v1/geocode/search"

In [37]:
response = requests.get(base_url, params=params).json()


In [38]:
lat = response["features"][0]["properties"]["lat"]
lon = response["features"][0]["properties"]["lon"]

In [39]:
base_url = 'https://api.geoapify.com/v2/places?'
categories = 'leisure'
radius = 10000
limit=500
params = {
    "categories": categories,
    "apiKey": apify_key,
    "limit": limit,
    "radius": radius
}
params["filter"] = f'circle:{lon},{lat},{radius}'
params["bias"] = f'proximity:{lon},{lat}'
leisure_dict = requests.get(base_url, params=params).json()


In [40]:
leisure_types = []
for feature in leisure_dict['features']:
    leisure_info = feature['properties']
    if 'city' in feature['properties']:
        leisure_cities = feature['properties']['city']
        if leisure_cities == 'Milton':
            leisure_types.append(leisure_info)

In [41]:
len(leisure_types)

234

In [42]:
school_df_milton.head(3)

,Index,Source_ID,Facility_Name,Facility_Type,Authority_Name,ISCED010,ISCED020,ISCED1,ISCED2,ISCED3,...,Postal_Code,PRUID,CSDNAME,CSDUID,Longitude,Latitude,Geo_Source,Provider,CMANAME,CMAUID
266,56dfa4e43f9d3e1fd2c1,342322,École élémentaire Dyane-Adam,Public,CS Viamonde,1,1,1,0,0,...,L9T6J4,35,Milton,3524009,-79.8787,43.51839,Source,Province of Ontario,Toronto,535
358,8beeab446e576b16b529,694632,ÉÉC Saint-Nicolas,Catholic,CS catholique MonAvenir,1,1,1,0,0,...,L9T4A3,35,Milton,3524009,-79.8776,43.52873,Source,Province of Ontario,Toronto,535
363,6866c0a2daa890c0e715,779909,ÉÉC Sainte-Anne,Catholic,CS catholique MonAvenir,1,1,1,0,0,...,L9T7V8,35,Milton,3524009,-79.8434,43.51934,Source,Province of Ontario,Toronto,535


In [48]:
leisure_data = {
    'City': ['Milton'],
    '# of Public Schools': [school_df_milton.loc[school_df_milton['Facility_Type'] == 'Public'].shape[0]],
    '# of Catholic Schools': [
        school_df_milton.loc[school_df_milton['Facility_Type'] == 'Catholic'].shape[0]],
    '# of Private Schools': [school_df_milton.loc[school_df_milton['Facility_Type'] == 'Private'].shape[0]],
    '# of Leisure Spaces': [len(leisure_types)
    ]}
milton_leisure_df = pd.DataFrame(leisure_data)
milton_leisure_df.set_index('City', inplace=True)
milton_leisure_df

,# of Public Schools,# of Catholic Schools,# of Private Schools,# of Leisure Spaces
City,,,,
Milton,21,14,4,234


In [46]:
cleaned_milton_df

Population  Household Income ($) Average Rent ($)  \
City   Year                                                      
Milton 2016    110128.0              120157.0           $2,821   
       2021    132979.0              142600.0           $2,785   

            Median Rent ($)  Estimated House Value ($)  Unemployment Rate (%)  \
City   Year                                                                     
Milton 2016          $2,150                   622911.0                    5.6   
       2021          $2,650                  1022000.0                   11.4   

             Property Tax Rate (%)  Rented Houses  Owned Houses  New Built  \
City   Year                                                                  
Milton 2016               0.006820         4840.0       29415.0        891   
       2021               0.007449         6950.0       33085.0       1073   

            Average Sale Price ($) Median Sale Price ($)  
City   Year                                               
Milton 2016               $657,525              $621,000  
       2021             $1,104,704              $990,000

In [49]:
school_df = school_public_df.loc[school_public_df['Provider'] == 'Province of Ontario']
school_df_Vaughan = school_public_df.loc[school_public_df['CSDNAME'] == 'Vaughan']
school_df_Vaughan

,Index,Source_ID,Facility_Name,Facility_Type,Authority_Name,ISCED010,ISCED020,ISCED1,ISCED2,ISCED3,...,Postal_Code,PRUID,CSDNAME,CSDUID,Longitude,Latitude,Geo_Source,Provider,CMANAME,CMAUID
279,692535cfd55370e2d395,302481,École élémentaire La Fontaine,Public,CS Viamonde,1,1,1,0,0,...,L0J1C0,35,Vaughan,3519028,-79.6211,43.83457,Source,Province of Ontario,Toronto,535
312,79e11a4048d78c9db6a0,739984,ÉIC Renaissance,Catholic,CS catholique MonAvenir,0,0,0,1,0,...,L6A1J5,35,Vaughan,3519028,-79.5042,43.84723,Source,Province of Ontario,Toronto,535
338,86f7798611be9a40291b,795917,ÉÉC Le-Petit-Prince,Catholic,CS catholique MonAvenir,1,1,1,1,0,...,L6A1Y3,35,Vaughan,3519028,-79.5246,43.85,Source,Province of Ontario,Toronto,535
7438,e672f922a3d6b04f8112,667420,Arts and Science Academy of Canada,Private,..,0,0,1,1,1,...,L4K2T4,35,Vaughan,3519028,-79.48250369,43.81830288,AGOL,Province of Ontario,Toronto,535
7456,2e7ee910b448134d012a,669532,Autumn Hill Academy,Private,..,0,0,1,1,0,...,L4K5M4,35,Vaughan,3519028,-79.4822664,43.83902177,AGOL,Province of Ontario,Toronto,535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13199,92b336d16276867e8736,952516,Westmount Collegiate Institute,Public,York Region DSB,0,0,0,1,1,...,L4J8G3,35,Vaughan,3519028,-79.4549,43.8163,Source,Province of Ontario,Toronto,535
13205,0336739f3094d122a32d,614700,Wilshire Elementary School,Public,York Region DSB,1,1,1,1,0,...,L4J7S8,35,Vaughan,3519028,-79.4668,43.81369,Source,Province of Ontario,Toronto,535
13208,809baf85b35e25858120,954292,Woodbridge College,Public,York Region DSB,0,0,0,1,1,...,L4L1J3,35,Vaughan,3519028,-79.5828,43.7799,Source,Province of Ontario,Toronto,535
13209,951f4b897eb9175075d0,617318,Woodbridge Public School,Public,York Region DSB,1,1,1,1,0,...,L4L1J7,35,Vaughan,3519028,-79.5956,43.78062,Source,Province of Ontario,Toronto,535
